0.18528336142008164### EXTRAS

In [28]:
import _pickle as cPickle
import pickle
import pandas as pd
file = open("generations",'rb')
gen = pickle.load(file)
test = pd.read_csv("test.csv")
len(gen)
file1 = open('hope_test.txt', 'r')
Lines = file1.readlines()
test_list = []
for _, line in enumerate(Lines):
    test_list.append(line.strip())
    
gen_new = []
for i in gen:
    words = i.split("_eou_")
    # print(words)
    # gen_new.append(" ".join(words[0:]))
    # print(words)
    if len(words) < 2:
        gen_new.append(words[0])
    else:
        t = "i asdf  an e amk eo i wero niwe waon oaoe ne nein oin: " + " ".join(words[0:])
        gen_new.append(" ".join(t.split(" ")))
test_list_new = []
for i in test_list:
    words = i.split("_eou_")
    test_list_new.append(" ".join(words[-1].split(" ")))
r = pd.DataFrame()
r["Response"] = gen_new[:-1]
r["Targets"] = test_list_new[1:]
r[1:].to_csv("results.csv")

In [24]:
r

,Response,Targets
0,Output: so what did you learn from all that? ...,It just got easier. At the beginning. It was ...
1,Output: feel like my stats were much higher t...,"habituation, habituation, right? Yeah. The fa..."
2,"Output: down, right? We're just not built to ...","I noticed that before doing the exposures, I ..."
3,"Output: I was really, really anxious. And the...",Right. Did that change by so yesterday?
4,"Output: kind of went down, like the anticipat...",Yeah.
...,...,...
2169,"Output: tomorrow, but I know what's gonna get...",it's really important because I Can't be cont...
2170,Output: get that done. It's important to know...,"by Jason, I'd like you to continue seeing a c..."
2171,"Output: by Jason, I'd like you to continue ...",The Counselor has been okay.
2172,Output: to continue seeing a counselor here. ...,"But that okay, it's been okay. But we'll cont..."


In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import warnings
warnings.filterwarnings('ignore')

import torch
import pandas as pd
import random
import math
import numpy as np
import nltk
from sklearn.metrics import classification_report
import logging
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from typing import Optional, Tuple
import transformers
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
import pickle
import torch.optim as optim
from sklearn import preprocessing
import wandb
torch.cuda.is_available()
import logging
device = torch.device("cpu")
# !pip install trl
# import trl.gpt2
import torch
import wandb
import time
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
from datasets import load_dataset

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import trl
from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
from trl.ppo import PPOTrainer
from trl.core import build_bert_batch_from_txt
def concat_ (a,b):
    new = []
    for i in range(len (a)):
        new.append(str(a[i])+': '+str(b[i]))
    return new


In [191]:
# df= pd.read_csv('data/switchboard_test.csv')

In [193]:
# df.Utterance = df["Text"]

In [ ]:

df1 = pd.read_csv("train_final1.csv")
# df2 = pd.read_csv('data/switchboard_test.csv')

# train_final = concat_(df1['DamslActTag'].to_list(),df1['Text'].to_list())
# test_final = concat_(df2['DamslActTag'].to_list(),df2['Text'].to_list())


from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

df_edited = df1
df_edited["Dialogue_Act"] = label_encoder.fit_transform(df_edited["Dialogue_Act"])
# df_edited["Utterance"] = df_edited["Text"]
print("We are working on:",device)

print("======================= WandB Login ===========================")
LOGGING = False
if LOGGING: wandb.init(project="gpt2_dac_response")

print("======================= Reading Data ===========================")
def concat_ (a,b):
    new = []
    for i in range(len (a)):
        new.append(str(a[i])+': '+str(b[i]))
    return new



counter = 0
label_encoder = preprocessing.LabelEncoder()
import torch
from torch import nn
from transformers import GPT2Model, GPT2LMHeadModel, AutoModel


from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Model, GPT2PreTrainedModel
from transformers import top_k_top_p_filtering
from transformers.modeling_outputs import ModelOutput
from torch import nn
from torch.nn import Identity
import torch.nn.functional as F
import torch
from dataclasses import dataclass
from typing import Optional, Tuple

In [3]:
df_edited = df_edited[["Utterance", "Dialogue_Act"]]

In [4]:
df= df_edited[["Utterance", "Dialogue_Act"]]
# df.to_csv("s_train.csv")

In [4]:
df = df_edited

In [5]:
df1 = pd.read_csv("train_final1.csv")
df1 

,Unnamed: 0.1,Unnamed: 0,ID,Type,Utterance,Dialogue_Act,Dialogue_Act_1,Emotion
0,0,0,1_0,P,<sos> >>P: I think I want to eat healthy. <eos>,id,NaN,NaN
1,1,1,1_1,P,<sos> >>P: I think I want to eat healthy. _eou...,id,NaN,NaN
2,2,2,1_2,P,<sos> >>P: I think I want to eat healthy. _eou...,id,NaN,NaN
3,3,3,1_3,T,<sos> >>P: I think I want to eat healthy. _eou...,irq,NaN,NaN
4,4,4,1_4,P,_eou_ <sos> >>P: I want to completely cut out ...,id,NaN,NaN
...,...,...,...,...,...,...,...,...
9415,9415,9415,99_40,T,"_eou_ <sos> >>P: And I think all last year, I ...",ynq,NaN,NaN
9416,9416,9416,99_41,P,"_eou_ <sos> >>T: Yeah, _eou_ <sos> >>P: it was...",id,NaN,NaN
9417,9417,9417,99_42,T,"_eou_ <sos> >>P: it was, it was better than 10...",id,NaN,NaN
9418,9418,9418,99_43,P,"_eou_ <sos> >>T: I, I think I've asked most mo...",id,NaN,NaN


In [6]:
utterances =[] 
dacs = []
for i in range(len(df.Utterance)):
    str = ""
    for j in range(4):
        curr = i-3+j
        if( j % 2 == 0):
            str = str + "Person1: "
        else:
            str = str + "Person2: "
        
        str = str + df.Utterance[max(0, curr)]
        str = str + " <eos> "
        if( j % 2 == 0):
            str = str + "Person2: "
        else:
            str = str + "Person1: "
            
    utterances.append(str)
    # dacs.append(df.Dialogue_Act[i])
        

In [4]:
df_new = pd.DataFrame()
df_new["Utterance"] = utterances
# df_new["Dialogue_Act"] = dacs

NameError: name 'utterances' is not defined

In [8]:
df_new.to_csv("test_new.csv")

In [ ]:
query = df.tokens[0].to(device)
# gpt2_model_ref.load_state_dict(torch.load("model.pt")) 
# query[0]
# gpt2_tokenizer.decode(
print(gpt2_tokenizer.decode(respond_to_batch(gpt2_model, torch.tensor([query.tolist()]).to(device), top_k=8, txt_len = 100)[0], skip_special_tokens = True))

In [9]:
# df
import sentence_transformers

from sentence_transformers import SentenceTransformer, util


print("======================= WandB Login ===========================")
LOGGING = True
if LOGGING: wandb.init(project="gpt2_dac_response")



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


======================= WandB Login ===========================


wandb: Currently logged in as: as3eem (use `wandb login --relogin` to force relogin)
/home/aseems/miniconda3/envs/myenv/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
model = GPT2LMHeadModel.from_pretrained("gpt2")

gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<sos>', eos_token='<eos>', pad_token='<pad>', eou_token='_eou_', speaker_one=">>P:", speaker_two=">>T:", truncation=True)
model.resize_token_embeddings(len(gpt2_tokenizer))

model.load_state_dict(torch.load("model.pt")) 

model.save_pretrained("GPT2_base")


model.load_state_dict(torch.load("model_20epoch.pt")) 

model.save_pretrained("GPT2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_model.resize_token_embeddings(len(gpt2_tokenizer))

gpt2_model.save_pretrained("GPT2_base")

gpt2_model = GPT2HeadWithValueModel.from_pretrained("GPT2_base", local_files_only = True).cpu()
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained("GPT2", local_files_only = True ).cpu()
# gpt2_model.load_state_dict(torch.load("model_train_checkpoint1120.pt")) 

# gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2', truncation=True)
# gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

gpt2_model_ref.resize_token_embeddings(len(gpt2_tokenizer))
# gpt2_tokenizer = GPT2Tokenizer.from_pretrained("GPT2Tokenizer")
# wandb.watch(gpt2_model, log='all')    
    

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at GPT2_base and are newly initialized: ['dac_head.l1.weight', 'dac_head.l0.bias', 'dac_head.l1.bias', 'v_head.summary.weight', 'dac_head.l0.weight', 'v_head.summary.bias', 'dac_head.l2.bias', 'dac_head.l2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at GPT2 and are newly initialized: ['dac_head.l1.weight', 'dac_head.l0.bias', 'dac_head.l1.bias', 'v_head.summary.weight', 'dac_head.l0.weight', 'v_head.summary.bias', 'dac_head.l2.bias', 'dac_head.l2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(50260, 768)

In [ ]:
!nvidia-smi

In [7]:
df_test = pd.read_csv("test.csv")

In [42]:
df_test

,ID,Type,Utterance,Dialogue_Act,Dialogue_Act_1,Emotion
0,100_0,T,"Okay, a great job on your homework was really ...",id,NaN,NaN
1,100_1,P,It just got easier. At the beginning. It was r...,id,NaN,NaN
2,100_2,T,"habituation, habituation, right? Yeah. The fac...",yq,NaN,NaN
3,100_3,P,"I noticed that before doing the exposures, I w...",id,NaN,NaN
4,100_4,T,Right. Did that change by so yesterday?,yq,NaN,NaN
...,...,...,...,...,...,...
2236,97_21,P,"Yeah, that would probably be helpful for him t...",id,NaN,NaN
2237,97_22,T,And you'd be okay saying release for him?,yq,NaN,NaN
2238,97_23,P,"Yes, yes.",op,NaN,NaN
2239,97_24,T,So I'll get you scheduled into see a counselor...,gt,NaN,NaN


In [72]:
last = "100"
utterance = []
response = []
sent = ""
for idx, row in df_test.dropna().iterrows():
    # print(row[2])
    curr = row["ID"].split("_")[0]
    if curr != last : 
        sent = ""
    sent = sent + row["Utterance"] + " _eou_"
    response.append(row["Utterance"])

    utterance.append(sent)
    last = curr
    

In [73]:
new_df = pd.DataFrame()
new_df["context"] = utterance[:-1]
new_df["response"] = response[1:]
new_df.to_csv("test_d.csv")

In [75]:
new_df['response'][2]

"I get what you're coming from but I I really did. I thought that I had it under control. And I didn't let myself get too far gone. I didn't want to be in how do I put this little Like, I want it to be in control. And I wanted to make sure that all my friends were Okay, so I didn't let myself drink too much."

In [77]:
new_df['context'][3]

"So Tommy I wanted to talk to because I was on Facebook the other day and saw some pictures that are kind of concerning to me of you and some of your friends at a party and you know, you're such a good kid, like, I don't know what you're doing drinking. It's good. ruin your life _eou_I mean it was just the like a social gathering it wasn't like I was looking for it the drinks were there and I have all my friends were doing it and it was just it's it's not something that I like do okay on occasion but whenever the opportunity presents itself it it's nice to just let loose and have a little bit of fun sometimes. _eou_I Tommy there's so many consequences with drinking you know you're not even 21 yet you could get arrested. You could you know have a drunk driving accident you could you know, mess up your schoolwork you have such a bright future ahead of you. It just really really concerns me. _eou_I get what you're coming from but I I really did. I thought that I had it under control. And 

In [38]:
keyword = i
# print(i)
keyword = " ".join(keyword.split()[-50:])
keyword = "this "
# print(keyword)
input_seq = keyword 
generated = torch.tensor(gpt2_tokenizer.encode(input_seq, return_token_type_ids=False)).unsqueeze(0)
generated = generated.to(device)
sample_outputs = model.generate(
                        generated, 
                        length_penalty = -10,
                        do_sample=False,   
                        top_k=9, 
                        min_length = 50,
                        max_length = 100,
                        top_p=0.7
                        # num_beams = 10
                        )
# print("==========================================")
t = (gpt2_tokenizer.decode(sample_outputs[0][len(keyword):], skip_special_tokens=False))
print(t)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_ What's going on? _eou_ Well, last time I started to tell you a little bit about one of my friends, my best friend Actually, she had this issue. So what happened was she went to a bathroom at our local library, and she found a video camera when she was going to the bathroom _eou_ in the bathroom? _eou_ Yeah, she actually, yeah. And she turned it into the police. And I guess


In [ ]:
responses = []
for i in utterance:
    input_seq = i
    generated = torch.tensor(gpt2_tokenizer.encode(input_seq, return_token_type_ids=False)).unsqueeze(0)
    generated = generated.to(device)
    sample_outputs = model.generate(
                            generated, 
                            length_penalty = -10,
                            do_sample=False,   
                            top_k=9, 
                            min_length = 50,
                            max_length = 100,
                            top_p=0.7
                            # num_beams = 10
                            )
    # print("==========================================")
    t = (gpt2_tokenizer.decode(sample_outputs[0][len(keyword):], skip_special_tokens=False))
    print(t)
    # t = interactive(" ".join(i.split()[-20:]), gpt2_tokenizer, model)
    # print(t)
    responses.append(t)

In [16]:

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
import argparse
import math
import numpy as np
from tqdm import tqdm
def interactive(input, tokenizer, model, top_k=9, ntok=20):
    """
    Generate and print out the response given input using the trained model
    :param input: an input string as prompt (i.e. How are you?)
    :param tokenizer: intialized tokenizer object for encoding the input
    :param model: the trained model to use for generate prediction
    :param top_k: number of samples for top_l sampling
    :param ntok: maximum number of tokens to generate
    Comment: Feed in the input to the model to generate the most probable
    token and concatenate it with current input.
    Continue this process iteratively until the model predicts the padding
    token or reach the maximum number of tokens.
    You may need to add the BOS token and special token to the input sentence
    before passing into model.
    Also, you may want to filter out your input sentence and meaningless tokens
    when printing out the response.
    """
    prompt = input

    # prompt = '<b> '+ pr
    
    enc = torch.tensor(tokenizer.encode(prompt, add_special_tokens=True)).unsqueeze(0).cpu()
    response = []

    while len(response) < ntok:
        with torch.no_grad():
            logits = model(enc).logits[0]
            # print(logits.size())
            # print(logits.size())
            #logits = logits.view(-1, logits.shape[1])
            topk, inds = torch.topk(logits, k=top_k)
            new_logits = Variable(torch.zeros(logits.size(0), logits.size(1))).cpu()
            logits = new_logits.scatter(1, inds, topk)
            
            probs = F.relu(logits[-1]) + 1e-9
            predicted = torch.multinomial(probs, 1)
            # next_token = torch.multinomial(probs, num_samples=1).squeeze(1)

            
            #new_word = predicted[-1].unsqueeze(0)
            
            if predicted.item() == tokenizer.pad_token_id:
                break
            else:
                # print(enc)
                # print(predicted)
                enc = torch.cat((enc[0], predicted), 0).unsqueeze(0)
                response.append(predicted.item())
                
    decoded = tokenizer.decode(response, skip_special_tokens=False)
    # print("Response:", decoded)
    return decoded



In [59]:
df3 = pd.read_csv("train.csv")

df["target"] = [str(x) for x in df3["Utterance"][1:]]
 
df

,Utterance,Dialogue_Act,target
0,<sos> >>P: I think I want to eat healthy. <eos>,5,I want to completely cut out fast food. I want...
1,<sos> >>P: I think I want to eat healthy. _eou...,5,I think I at least needs to go to the grocery....
2,<sos> >>P: I think I want to eat healthy. _eou...,5,Is that something that you haven't been doing?
3,<sos> >>P: I think I want to eat healthy. _eou...,6,"No, not lately. I've just been grabbing food a..."
4,_eou_ <sos> >>P: I want to completely cut out ...,5,"Like, I just haven't had the energy or desire ..."
...,...,...,...
9415,"_eou_ <sos> >>P: And I think all last year, I ...",10,"Well, I think if you can do something for some..."
9416,"_eou_ <sos> >>T: Yeah, _eou_ <sos> >>P: it was...",5,right? To raise money for ALzheimer's without ...
9417,"_eou_ <sos> >>P: it was, it was better than 10...",5,"Yeah. Wow. Yeah. Yeah, that should be good. no..."
9418,"_eou_ <sos> >>T: I, I think I've asked most mo...",5,Thank you.


In [88]:
df_batch = df.sample(30)
inputs = df_batch.Utterance.tolist()
target = df_batch.target.tolist()
dac = df_batch.target.tolist()

for i in inputs:
    print(i.split()[-10:])
    out = interactive(" ".join(i.split()[-10:]), gpt2_tokenizer, gpt2_model, top_k = 5)
    print("++"*10, out)
    outputs.append(out)
# sent = "_eou_ <sos> >>T: Exciting ? _eou_ <sos> >>P: Yeah, it definitely is. I can do that. _eou_ <sos> >>T: So I will see you and next week ? _eou_ <sos> >>P: okay <eos>',"
# interactive(sent+sent+sent + sent, gpt2_tokenizer, gpt2_model, top_k = 5)

['score', 'would', 'you', 'actually', 'give', 'it', 'in', 'the', 'end?', '<eos>']
++++++++++++++++++++ <eos> <eos> <sos> <eos> <sos> <eos> <sos> <sos> <eos> <eos> <sos> <eos> <sos> <eos> <eos> <sos> <eos>
['those', 'two', 'and', 'respect,', 'respect.', '_eou_', '<sos>', '>>T:', 'Okay,', '<eos>']
++++++++++++++++++++ <eos> <eos> <sos> <sos> <sos> <eos> <sos> <sos> <eos> <sos> <eos> <sos> <eos> <eos> <sos> <sos> <eos> <eos> <eos> <eos> <eos> <sos> <sos> <sos>
['I', 'mean,', "there's", 'so', 'many', 'things', "it's", "it's", 'everywhere.', '<eos>']
++++++++++++++++++++ <eos> <eos> <sos> <eos> <sos> <sos> <eos> <eos> <sos> <eos> <sos> <eos> <eos> <sos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos> <eos>
['_eou_', '<sos>', '>>T:', 'you', 'have', 'a', 'craving', 'for', 'marijuana', '<eos>']
++++++++++++++++++++ <sos> <eos> <eos> <eos> <eos> <eos> <sos> <eos> <sos> <sos> <eos> <sos> <eos> <eos>
['thinking', 'that', 'might', 'be', 'a', 'better', 'way', 'to', 'go?', '<eos>']
+++++++++++++++++

In [51]:
model = gpt2_model
tokenizer = gpt2_tokenizer

In [66]:
df_batch.to_csv("n.csv")

In [89]:
def eval_keywords(keywords):
    outputs=[]
    model.eval()
    for keyword in tqdm(keywords):
        keyword = " ".join(keyword.split()[-20:])
        input_seq = "<sos> " + keyword + " _eou_ "
        generated = torch.tensor(tokenizer.encode(input_seq, return_token_type_ids=False)).unsqueeze(0)
        generated = generated.to(device)
        # sample_outputs = model.generate(
        #                             generated, 
        #                             do_sample=True,   
        #                             top_k=30, 
        #                             max_length = 50,
        #                             top_p=0.90, 
        #                             num_return_sequences=2
        #                             )
        sample_outputs = model.generate(
                                    generated, 
                                    min_length= 20,
                                    max_length = 80,
                                    )
        # print("\n\n ====== \n")
        # print(input_seq)
        # print("\n -- -- \n")

        # print(sample_outputs)
        # print("--- ---")
        # print(sample_outputs[0])
        # print("--- ---")

        # print(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))
        # print(tokenizer.decode(sample_outputs[1], skip_special_tokens=True))
        
        outputs.append(tokenizer.decode(sample_outputs[0], skip_special_tokens=True))        
    return outputs


In [90]:
new = eval_keywords(inputs)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:42<00:00,  1.43s/it]


In [70]:
df_b = pd.read_csv("new.csv")
# df_b = df_b[["target","predicted_rac",""]]
predicted = df_b.predicted_rac

In [ ]:
df_batch

In [91]:
df_fin = pd.DataFrame()
df_fin["Utterance"] = inputs[:-1]
df_fin["Generated"] = new[:-1]
df_fin["Target"] = target[:-1]
# df_fin["dac_next"] = df_batch.reset_index(drop=True).Dialogue_Act.tolist()[1:]
# df_fin["predicted_rac"] = predicted
df_fin.to_csv("ne2.csv")

In [79]:
df_fin

,Utterance,Generated,Target,dac_next,predicted_rac
0,"_eou_ <sos> >>P: walk off, okay or he will be...",in your life that you can talk to. What do yo...,"And I think also, he understood why I did it. ...",3,3
1,"<sos> >>T: So Steph, I know you just want to g...",">>T: Hello, Peggy, how you doing today? _eou...",I don't really feel like filling out a plan. I...,5,5
2,_eou_ <sos> >>T: Yeah. any substance related p...,change and it's very important to you that yo...,Okay. And we'll talk about any type of legal p...,0,3
3,"_eou_ <sos> >>P: Well, alcohol definitely impa...",>>T: Jasmine how you doing today? _eou_ >>P...,Yeah.,6,1
4,"_eou_ <sos> >>P: Yeah, I have some pills in my...",versus just not just a friend's house like an...,"Um, you know, I want to make sure that all the...",1,5
5,_eou_ <sos> >>T: Do you feel the trick good in...,back. _eou_ >>T: So you weren't you weren't ...,How would you describe your mood? Most of the ...,0,6
6,_eou_ <sos> >>T: And how much do you believe t...,went anyway. _eou_ >>T: So was it after he h...,And you believe them now. Right? So try to kee...,6,5
7,_eou_ <sos> >>P: It wasn't that long. She was ...,"late, but I'm late enough. _eou_ >>T: Have y...","No, I just stopped going",10,5
8,"_eou_ <sos> >>P: Yes, I do. Yeah. Yeah. Well, ...",>>T: But eventually you're going to come to o...,And then I'm Facebook friends when they go hom...,1,5
9,"<sos> >>T: between here and there, obviously, ...",around. _eou_ >>T: So mainly just kind of ha...,What are you gonna do if I fail?,3,6


In [ ]:
config = {
    # "lm_name": "/content/drive/MyDrive/MWP_gpt2_paraphrases_512",
    # "ref_lm_name": "/content/drive/MyDrive/MWP_gpt2_paraphrases_512",
    # "cls_model_name": "/content/drive/MyDrive/question_paraphrasing_semantic_similarity_models/ParaQD_v3.1",
    "tk_name": "gpt2",
    "steps": 51200,
    "batch_size": 18,
    "forward_batch_size": 9,
    "ppo_epochs": 5,   
    "txt_in_len": 15,
    "txt_out_len": 50,
    "lr": 1.41e-4,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
}


In [ ]:
_ = gpt2_model.to(device)
_ = gpt2_model_ref.to(device)

In [ ]:
!pip install rouge-metric
bhn

In [ ]:
import nltk
import evaluate 
# import rouge
nltk.download('punkt')

# model_sentence = SentenceTransformer('all-MiniLM-L6-v2')

def get_diversity(actual, generated):
    generated = generated.split(" ")
    actual = actual.split(" ")
    # rouge = evaluate.load('rouge')
    # results_r = rouge.compute(predictions=[sentence1],
    #                      references=[sentence2])
    
    
    results = nltk.translate.bleu_score.sentence_bleu([actual], generated, weights = [1]) 
    return results

def get_reward(sentence_1, sentence_2):
    lambda1 = 1
    # semantic_preservation = score_biencoder(model, sentence_1, sentence_2)
    sentences = [sentence_1, sentence_2]
    # paraphrase_score = util.paraphrase_mining(model, sentences)[0][0]
    
    print("Gnerated: ", sentence_1)
    print("Target: ", sentence_2)
    diversity = get_diversity(sentence_2, sentence_1)
    # print(lambda1 * diversityget_re
    print(lambda1 * (diversity + paraphrase_score))
    return lambda1 * (diversity + paraphrase_score)
            

In [ ]:
gpt2_tokenizer.eos_token_id

In [ ]:
df["query"] = df.Utterance
df3 = pd.read_csv("train.csv")

df["target"] = [str(x) for x in df3["Utterance"][1:]]
labels = [int(x) for x in df["Dialogue_Act"][1:]]
labels.append(0)

df["dac_labels"] = labels


In [ ]:
df["tokens"] = [torch.tensor(gpt2_tokenizer.encode_plus(str(x), \
                max_length=300, pad_to_max_length=True, add_special_tokens=True, truncation = True, return_attention_mask=True)["input_ids"]
) for x in df["Utterance"]]

In [144]:
!nvidia-smi

Thu Aug 11 17:10:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  RTX A6000           On   | 00000000:41:00.0 Off |                  Off |
| 39%   69C    P2    96W / 300W |  14267MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  RTX A6000           On   | 00000000:61:00.0 Off |                  Off |
| 31%   

In [ ]:
# # import 
# gen_kwargs = {
#     "min_length":20,
#     "top_k": 8,
#     "top_p": 0.9,
#     "do_sample": True,
#     "pad_token_id": gpt2_tokenizer.pad_token_id
# }
# import warnings
# warnings.filterwarnings("ignore")
# import re
# # from quantulum3 import parser
# import pandas as pd
# import numpy as np
# # import requests
# # import json
# # from nltk.corpus import wordnet
# # from nltk.tokenize import word_tokenize
# # import nltk
# # nltk.download('wordnet')
# # nltk.download('punkt')
# # import inflect
# torch.autograd.set_detect_anomaly(True)

# # from similarity.normalized_levenshtein import NormalizedLevenshtein
# import torch
# import logging
# # from sklearn.feature_extraction.text import TfidfVectorizer
# import random
# import pickle
# from scipy.spatial.distance import cdist
# from sentence_transformers import SentenceTransformer

# ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, gpt2_tokenizer,  **config)
# fbs = config['forward_batch_size']
# gen_len = 15
# for epoch in tqdm(range(int(np.ceil(config["steps"]/config['batch_size'])))):
#     # torch.cuda.empty_cache()
#     logs = dict()
#     game_data = dict()
#     timing = dict()
#     t0 = time.time()
    
#     #### get a batch from the dataset
#     df_batch = df.sample(config['batch_size'])
#     game_data['query'] = df_batch['query'].tolist()
#     game_data["target"] = df_batch["target"].tolist()
#     # tens = torch.tensor(df_batch['tokens'].tolist()).to(device
#     tens = []
#     for i in df_batch['tokens']:
#         tens.append(torch.tensor(i).to(device))
#     # query_tensors = torch.stack(df_batch['tokens'].tolist())
#     query_tensors = torch.stack(tens)
#     dac_labels = torch.tensor(df_batch["dac_labels"].tolist()).to(device)
#     #### get response from gpt2
#     t = time.time()
#     total_length = config['txt_in_len']+config['txt_out_len']
#     response_tensors = []
#     for i in range(int(config['batch_size']/fbs)):
#         response  = respond_to_batch(gpt2_model, query_tensors[i*fbs:(i+1)*fbs],
#                                      txt_len= 50, top_k = 8)
#         response_tensors.append(response)
#         # for j in range(i*fbs,(i+1)*fbs):
#         # response = gpt2_model.generate(query_tensors[i*fbs:(i+1)*fbs], 
#         #                                max_new_tokens=gen_len, **gen_kwargs, size_penalty = -10)
#         # response_tensors.append(response.squeeze()[-gen_len:])
#         # response_tensors.append(response)
#     response_tensors = torch.cat(response_tensors).to(device)
#     game_data['response'] = [gpt2_tokenizer.decode(response_tensors[i, :],  skip_special_tokens=True) for i in range(config['batch_size'])]
#     timing['time/get_response'] = time.time()-t

#     #### tokenize text for sentiment analysis
#     t = time.time()
#     # texts = [q + r for q,r in zip(game_data['query'], game_data['response'])]

#     # sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
#     timing['time/build_input_sentiment'] = time.time()-t

#     #### get sentiment score
#     t = time.time()
#     rewards = []
#     for i in range(int(config['batch_size']/fbs)):
#         # res = sentiment_model.forward(sentiment_inputs[i*fbs:(i+1)*fbs],
#         #                               attention_masks[i*fbs:(i+1)*fbs])[0][:, 1].detach()
#         for sentence1, sentence2 in zip(game_data['response'][i*fbs:(i+1)*fbs],game_data['target'][i*fbs:(i+1)*fbs]):
#             res = get_reward(model_sentence, sentence1,sentence2)
#             rewards.append(torch.tensor([res]))
#     rewards = torch.cat(rewards).to(device)
#     timing['time/get_sentiment_preds'] = time.time()-t
    
#     #### Run PPO training 
#     t = time.time()
#     stats = ppo_trainer.step(query_tensors, response_tensors, rewards, dac_labels)
#     timing['time/optimization'] = time.time()-t
     
#     #### Log everything
#     timing['time/epoch'] = time.time()-t0
#     table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]
#     logs.update({'game_log':wandb.Table(
#         columns=['query', 'response', 'reward'],
#         rows=table_rows)})
#     logs.update(timing)
#     logs.update(stats)
#     logs['env/reward_mean'] = torch.mean(rewards).cpu().numpy()
#     logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
#     logs['env/reward_dist'] = rewards.cpu().numpy()
#     torch.save(gpt2_model.state_dict(), "model_trained_checkpoint1.pt")
#     # print(logs)
#     wandb.log(logs)

In [ ]:
df_edited.Utterance[2]

In [ ]:
# total_loss:  tensor(0.7857, device='cuda:1', grad_fn=<AddBackward0>)
# LM_Loss:  tensor(0.6318, device='cuda:1', grad_fn=<NllLossBackward0>) 
# Loss_p:  tensor(0.1523, device='cuda:1', grad_fn=<MeanBackward0>) 
# Loss_v:  tensor(16.0353, device='cuda:1', grad_fn=<MulBackward0>)


In [ ]:
df

In [ ]:
torch.save(gpt2_model.state_dict(), "model_trained1.pt")
# gpt2_model.load_state_dict(torch.load("model_trained.pt")) 
# gpt2_model_ref 

In [ ]:
#### get a batch from the dataset
bs = 2
game_data = dict()
df_batch = df.sample(bs)
game_data['query'] = df_batch['query'].tolist()
query_tensors = torch.stack(df_batch['tokens'].tolist()).to(device)

#### get response from gpt2 and gpt2_ref
total_length = config['txt_in_len']+config['txt_out_len']
response_tensors_ref  = respond_to_batch(gpt2_model, query_tensors, None, 10)
game_data['Response'] = [gpt2_tokenizer.decode(response_tensors_ref[i, :], skip_special_tokens = True) for i in range(bs)]

response_tensors  = gpt2_model_ref.generate(query_tensors)
game_data["Targets"]= df_batch["target"]
game_data['response (after)'] = [gpt2_tokenizer.decode(response_tensors[i, :], skip_special_tokens = True) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
# texts = [q + r for q,r in zip(game_data['query'], game_data['response (before)'])]
# sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
# rewards = sentiment_model.forward(sentiment_inputs, attention_masks)[0][:, 1].detach()
# game_data['rewards (before)'] = rewards.cpu().numpy()

# texts = [q + r for q,r in zip(game_data['query'], game_data['response (after)'])]
# sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
# rewards = sentiment_model.forward(sentiment_inputs, attention_masks)[0][:, 1].detach()
# game_data['rewards (after)'] = rewards.cpu().numpy()
# DacHead
# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

In [ ]:
df_results.to_csv("results.csv")

In [ ]:
!nvidia-smi